# Tricky Functions and Cautionary Tales, Continued

In the previous tutorial, we talked about discontinuities (functions that aren't $C_1$-continuous) as a common source of optimization problems.

Let's talk about another: NaNs.

NaNs are a familiar and terrifying sight to any battle-hardened computationalist.

What's a NaN? It's the value that gets returned whenever a function gets evaluated anywhere outside of its domain. When do NaNs tend to appear? Several functions are common culprits:

1. $\sqrt x$ if $x < 0$. (Note: `aerosandbox.numpy` will evaluate this fine, but it can't trace automatic derivatives through complex math, so don't use this in optimization and try to avoid it in general.)

2. More generally, any $x^y$ where $y$ is non-integer and $x < 0$.

3. $\ln x$ or $\log x$ evaluated at $x < 0$.

4. Infinities in either a function's value or its derivative.

The problem with NaNs in optimization is twofold:

1. We can't get a value for the objective function or constraint that is returning the NaN, so we don't know "how good we're doing".

2. We can't get a gradient of the NaN-returning function, so we don't know which direction to look in order to return to the well-posed space.

So, stay away from NaNs! This may seem easy, but it's much easier to introduce NaNs than one might imagine. Let's look at some examples:

## Example 1: Unconstrained $\sqrt x$ Minimization

Let's start with an example that should obviously fail. We're going to try minimizing $\sqrt x$, starting with an initial guess of $x = -1$. Because our initial point will return a NaN, this will obviously fail to solve:

In [25]:
import aerosandbox as asb
import aerosandbox.numpy as np

opti = asb.Opti()

x = opti.variable(init_guess=-1)

opti.minimize(np.sqrt(x))

try:
    sol = opti.solve()
except RuntimeError as e:
    print(e)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints

CasADi -  WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]


As expected, this failed, reporting that "Invalid number in NLP (NonLinear Program) function or derivative detected".

Okay, so what could we do better? Well, let's make two changes:

1. Let's start from an initial guess of $x=1$, which will return a non-NaN value.
2. Let's implement a constraint $x > 0$, which should keep the function $\sqrt x$ from going into NaN territory.

Now, this optimization problem should be well-posed, right?

In [26]:
opti = asb.Opti()

x = opti.variable(init_guess=1, lower_bound=0)

opti.minimize(np.sqrt(x))

try:
    sol = opti.solve()
except RuntimeError as e:
    print(e)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.

Nope! Technically this optimization problem is well-posed, but it is ill-posed for a gradient-based numerical solution, and therefore it fails to solve. Why?

Because at $x=0$, the derivative of $\sqrt(x)$ approaches infinity - this creates a NaN.

So let's make another change. Instead of minimizing $\sqrt x$, let's minimize $x ^ {1.5}$. This function has a derivative that approaches zero in the $x \rightarrow 0$ limit. Let's see what happens:

In [27]:
opti = asb.Opti()

x = opti.variable(init_guess=1, lower_bound=0)

opti.minimize(x ** 1.5)

sol = opti.solve()

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This solves - nice!

One thing to know on this note: IPOPT does something it calls "bounds pushing" - this is detailed in the excellent original IPOPT paper by Andreas Waechter, but the basic idea is that IPOPT solver performance seems to improve if the feasible space is extended by a small amount ($\epsilon \approx 10^{-8}$). In cases like this, that has the potential to allow a function like $x^{1.5}$ bounded by $x>0$ to go very slightly negative and NaN. IPOPT has lots of tools to help get the iterate back into the well-posed design space, but just be aware of this possibility.

Let's talk about one last possibility, and an issue that can come up with conditionals.

Imagine we have the problem:

* Minimize $f(x)$, where
* $f(x) = \begin{cases}
x^{1.5} & \text{for } x\geq 0\\
(-x)^{1.5} & \text{for } x < 0 \\
\end{cases}
$

Note that $f(x)$ is a $C_1$-continuous function - both its value and derivative are continuous everywhere. Also, note that $f(x)$ should never return NaN for any input value $x$.

Let's see what happens when we try to solve this:

In [28]:
opti = asb.Opti()

x = opti.variable(init_guess=1)

f = np.where(
    x >= 0,
    x ** 1.5,
    (-x) ** 1.5
)

opti.minimize(f)

try:
    sol = opti.solve()
except RuntimeError as e:
    print(e)

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints

CasADi -  WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi -  WARNING("solver:nlp_grad_f failed: NaN detected for output grad_f_x, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]


This errored too! Why?

Because the *intermediate values* of $x^{1.5}$ in `np.where()` are NaN. In the current functionality of `np.where()`, if either value is NaN, the result is NaN (regardless of the value of `condition`) - this will hopefully be fixed in the future.

For now, you can bypass this problem by using `np.fabs(x)` or `np.fmax(x, 0)` to ensure that each piece of the conditional is non-NaN (even for values that will never be used):

In [29]:
opti = asb.Opti()

x = opti.variable(init_guess=1)

f = np.where(
    x > 0,
    np.fmax(x, 0) ** 1.5,
    np.fabs(x) ** 1.5
)

opti.minimize(f)


sol = opti.solve()

print(f"x = {sol.value(x)}")

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 1

This works, and it indicates that the optimum is `x = 0` as expected.